<a href="https://colab.research.google.com/github/Weff24/PrivacyPlate/blob/main/pp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Extraction from XML Files

In [46]:
from google.colab import drive
drive.mount('/content/drive')

import xml.etree.ElementTree as ET
import os
import sys
from collections import defaultdict

from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import matplotlib.patches as patches

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. XML Data Extraction: License Plate Crop Borders

In [ ]:
# Path to annotation folder
annot_dir_path = Path("/content/drive/MyDrive/01-License Plates/annotations")

if not annot_dir_path.exists():
  print("Annotation directory path does not exist")
  sys.exit(1)

# Extract the box from xml files
box_locations = defaultdict(list)
box_elements = ['xmin', 'ymin', "xmax", "ymax"] # order of the box location

for path in os.listdir(annot_dir_path):
    tree = ET.parse(f'{annot_dir_path}/{path}')  # Replace 'your_xml_file.xml' with your XML file path
    root = tree.getroot()

    # Get key from XML
    xml_ind =int(path[4:-4])

    for element in box_elements:
      box_locations[xml_ind].append(int(root.find('object').find('bndbox').find(element).text))

# 2. License Plate Image Data Import

In [ ]:
img_dir_path = Path("/content/drive/MyDrive/01-License Plates/images")

if not img_dir_path.exists():
  print("Image directory path does not exist")
  sys.exit(1)

images = []

i = 0
for i, path in enumerate(os.listdir(img_dir_path)):
  image = mpimg.imread(f"{img_dir_path}/{path}")
  fig, ax = plt.subplots()

  ax.imshow(image)

  # Red box around to see the license plate
  box = box_locations[int(path[4:-4])]
  rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='r', facecolor='none')
  ax.add_patch(rect)

  plt.show()


  i += 1
  if i > 5:
    break

# 3. CNN Construction

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')